In [1]:
import pandas as pd

df = pd.read_csv("names_dataset.csv")

# Estructura
print(df.head())
print(df.shape)

# Unicidad
print("Nombres únicos:", df['Full Name'].nunique())

# Longitud de strings
print(df['Full Name'].str.len().describe())



   ID             Full Name
0   1         María Sánchez
1   2          Marta Alonso
2   3       Javier González
3   4    Carmen López López
4   5  Isabel Moreno Moreno
(5000, 2)
Nombres únicos: 3016
count    5000.000000
mean       17.314000
std         4.644967
min         8.000000
25%        13.000000
50%        17.000000
75%        21.000000
max        33.000000
Name: Full Name, dtype: float64


El dataset provisto (names_dataset.csv) contiene:

* 5.000 registros

* 2 columnas: ID y Full Name

* Nombres completos en español, con:

* Acentos

* Nombres y apellidos compuestos

* Cantidad variable de tokens

* Ausencia de estructura explícita (nombre / apellido)

Por lo tanto, el problema corresponde a matching textual, no a análisis semántico.

En primer lugar comprobamos si es conveniente realizar una separación previa de los datos en nombre y apellido dado que comunmente los sistemas de información tienen esta estructura. Además la ponderación mayor al apellido podría ayudar a enfocar más el algoritmo, sin embargo dado que no hay una estructura confiable (el dataset no viene separado previamente ni con un separador) separar nombre y apellido genera un peor score. 

# Alternativas evaluadas
## A. Comparación sobre el nombre completo

Este enfoque consiste en trabajar directamente sobre el nombre completo tal como aparece en el dataset. Previamente, los textos se normalizan (conversión a minúsculas y eliminación de acentos) para evitar diferencias superficiales.
La comparación se realiza sobre el string completo utilizando técnicas de similaridad basadas en tokens y distancia de edición, lo que permite capturar variaciones de orden, errores de tipeo y omisiones parciales sin asumir una estructura fija del nombre.

## B. Separación heurística de nombre y apellido

En esta alternativa se intenta dividir automáticamente el nombre completo en dos partes: nombre y apellido, asumiendo que el último token corresponde al apellido y el resto al nombre.
Luego, se calcula la similitud de cada componente por separado y se obtiene un score final ponderado, asignando mayor peso al apellido. Este enfoque busca priorizar coincidencias de apellido, aunque depende fuertemente de una heurística que no siempre se cumple en nombres reales.

# Experimento empírico

Se realizaron pruebas con consultas realistas contra el dataset, comparando los resultados obtenidos con ambos enfoques. El objetivo fue evaluar no solo el score numérico, sino también la calidad del ranking y la coherencia de las coincidencias devueltas.

# Resultados observados

El enfoque basado en el nombre completo mostró resultados más precisos y consistentes en la mayoría de los casos.
Por el contrario, la separación heurística introdujo varios problemas: aumentó la cantidad de falsos positivos, sobreponderó coincidencias parciales de apellido y perdió información relevante en nombres compuestos. En múltiples escenarios, esto provocó que el resultado correcto quedara peor posicionado en el ranking.


| Query             | Mejor match (Full) | Score    | Mejor match (Split)      | Score    |
| ----------------- | ------------------ | -------- | ------------------------ | -------- |
| Maria Sanchez     | María Sánchez      | 100      | María Sánchez            | 100      |
| Juan Carlos Perez | Juan Pérez         | 100      | Juan Pérez               | 100      |
| Luis Gonzalez     | Luis González      | 100      | **Luis Martín González** | 100      |
| Ana Lopez         | Ana Sánchez López  | 100      | **Ana Jiménez López**    | 100      |
| Pedro Ramirez     | Pedro Ruiz         | **78.2** | Pedro Álvarez            | **74.3** |

In [4]:
%pip install rapidfuzz

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\pbonafe\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [6]:
import pandas as pd
import unicodedata
from rapidfuzz import fuzz

# Cargar dataset
df = pd.read_csv("names_dataset.csv")

def normalize(text: str) -> str:
    text = text.lower()
    text = unicodedata.normalize("NFKD", text)
    text = "".join(c for c in text if not unicodedata.combining(c))
    return " ".join(text.split())

df["normalized_name"] = df["Full Name"].apply(normalize)

#enfoque A
def similarity_full(query: str, candidate: str) -> float:
    return fuzz.token_set_ratio(query, candidate)

#enfoque B
def split_name(full_name: str):
    tokens = full_name.split()
    if len(tokens) == 1:
        return tokens[0], ""
    return " ".join(tokens[:-1]), tokens[-1]

def similarity_split(query: str, candidate: str, w_name=0.4, w_surname=0.6) -> float:
    q_name, q_surname = split_name(query)
    c_name, c_surname = split_name(candidate)

    score_name = fuzz.token_set_ratio(q_name, c_name) if q_name and c_name else 0
    score_surname = fuzz.token_set_ratio(q_surname, c_surname) if q_surname and c_surname else 0

    return score_name * w_name + score_surname * w_surname


#prueba con datos sintéticos
queries = [
    "Maria Sanchez",
    "Juan Carlos Perez",
    "Luis Gonzalez",
    "Ana Lopez",
    "Pedro Ramirez"
]

results = []

for q in queries:
    q_norm = normalize(q)

    for _, row in df.iterrows():
        full_score = similarity_full(q_norm, row["normalized_name"])
        split_score = similarity_split(q_norm, row["normalized_name"])

        results.append({
            "query": q,
            "id": row["ID"],
            "candidate": row["Full Name"],
            "full_score": round(full_score, 2),
            "split_score": round(split_score, 2)
        })

#Tabla comparativa final
results_df = pd.DataFrame(results)





In [9]:
required_cols = {"query", "candidate", "full_score", "split_score"}
missing = required_cols - set(results_df.columns)
if missing:
    raise ValueError(f"results_df no tiene las columnas requeridas: {sorted(missing)}")

# 1) Mejor match por enfoque FULL (máximo full_score por query; desempate estable)
best_full = (
    results_df
    .sort_values(["query", "full_score", "id"], ascending=[True, False, True], kind="mergesort")
    .drop_duplicates(subset=["query"], keep="first")
    .rename(columns={"candidate": "candidate_full", "full_score": "full_score"})
    .loc[:, ["query", "candidate_full", "full_score"]]
)

# 2) Mejor match por enfoque SPLIT (máximo split_score por query; desempate estable)
best_split = (
    results_df
    .sort_values(["query", "split_score", "id"], ascending=[True, False, True], kind="mergesort")
    .drop_duplicates(subset=["query"], keep="first")
    .rename(columns={"candidate": "candidate_split", "split_score": "split_score"})
    .loc[:, ["query", "candidate_split", "split_score"]]
)

# 3) Tabla comparativa final (lado a lado)
comparison = best_full.merge(best_split, on="query", how="inner")

comparison


,query,candidate_full,full_score,candidate_split,split_score
0,Ana Lopez,Ana López López,100.00,Ana López López,100.00
1,Juan Carlos Perez,Carlos Pérez,100.00,Carlos Pérez,100.00
2,Luis Gonzalez,Dr. Luis González,100.00,Dr. Luis González,100.00
3,Maria Sanchez,María Sánchez,100.00,María Sánchez,100.00
4,Pedro Ramirez,Pedro Ruiz,78.26,Pedro Romero Álvarez,74.29


# Conclusión

Separar nombre y apellido sin contar con una estructura confiable en los datos introduce heurísticas frágiles que terminan degradando la precisión del matching.
En este dataset en particular, no existe una garantía semántica de que el último token represente correctamente el apellido ni de que los tokens restantes correspondan al nombre. Asumir esa estructura genera errores sistemáticos difíciles de corregir.

El enfoque más robusto, explicable y defendible consiste en trabajar sobre el nombre completo, aplicando tokenización y técnicas de comparación textual tolerantes a errores, sin forzar divisiones artificiales.

# Estrategia final adoptada

La estrategia elegida evita la creación de campos artificiales de nombre y apellido y se basa en la normalización del texto y el uso de métricas de similaridad que combinan comparación por tokens y distancia de edición.
Los resultados se filtran según un umbral configurable y se ordenan de mayor a menor similitud.

Este enfoque reduce falsos positivos, no depende de reglas arbitrarias, es fácil de explicar en una entrevista técnica y escala adecuadamente para el tamaño del dataset analizado.